In [ ]:
!pip install torchattack > /dev/null
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as DataUtils
from torchvision.models import squeezenet1_0
from torchattack import PGD
import os
import numpy as np
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=0)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, padding=0)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class SqueezeNetMNIST(nn.Module):
    def __init__(self):
        super(SqueezeNetMNIST, self).__init__()
        self.model = squeezenet1_0(num_classes=10)
        self.model.features[0] = nn.Conv2d(1, 96, kernel_size=7, stride=2)
        self.model.classifier[1] = nn.Conv2d(512, 10, kernel_size=1)

    def forward(self, x):
        return self.model(x)

In [ ]:
def get_dataloader(train=False, batch_size=64):
    transform = transforms.Compose([transforms.ToTensor()])
    dset = datasets.MNIST(root='./data', train=train, download=True, transform=transform)
    return DataUtils.DataLoader(dset, batch_size=batch_size, shuffle=train)

def train_proxy_model(arch, epochs=2):
    print(f"Training Proxy {arch}...")
    model = LeNet5().to(device) if arch == 'lenet' else SqueezeNetMNIST().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    train_loader = get_dataloader(train=True)
    
    model.train()
    for epoch in range(epochs):
        for imgs, lbls in train_loader:
            imgs, lbls = imgs.to(device), lbls.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, lbls)
            loss.backward()
            optimizer.step()
    return model

In [ ]:
base_path = '/content/drive/My Drive/adversarial_mnist'
dirs = {
    'weights': f'{base_path}/model_weights',
    'std_adv': f'{base_path}/large_adversarial_examples',
    'proxy_adv': f'{base_path}/proxy_adversarial_examples'
}
for d in dirs.values(): os.makedirs(d, exist_ok=True)

target_models = [
    ('lenet', 'lenet.pth'),
    ('lenet', 'lenet_robust.pth'),
    ('squeezenet', 'squeezenet.pth'),
    ('squeezenet', 'squeezenet_robust.pth')
]

Processing resnet18_pgd_robust.pth...
  Saved 500 examples
  Average L2 Perturbation: 5.0648

Processing resnet18_standard_trained.pth...
  Saved 500 examples
  Average L2 Perturbation: 4.4346

Processing resnet50_pgd_robust.pth...
  Saved 500 examples
  Average L2 Perturbation: 4.8592

Processing resnet50_standard_trained.pth...
  Saved 500 examples
  Average L2 Perturbation: 4.2504



In [ ]:
loader = get_dataloader(train=False, batch_size=32)

print("\n--- Generating Standard Attacks (Source = Target) ---")
for arch, filename in target_models:
    print(f"Processing {filename}...")
    model = LeNet5().to(device) if 'lenet' in arch else SqueezeNetMNIST().to(device)
    model.load_state_dict(torch.load(f"{dirs['weights']}/{filename}", map_location=device))
    model.eval()
    
    adversary = PGD(model, eps=0.3, steps=40, random_start=True)
    
    examples, true_lbls, fooled_lbls, dists = [], [], [], []
    total, fooled = 0, 0
    
    for img, lbl in loader:
        if len(examples) >= 500: break
        img, lbl = img.to(device), lbl.to(device)
        adv = adversary(img, lbl)
        
        with torch.no_grad():
            pred = model(adv).argmax(1)
        
        batch_fool = (pred != lbl)
        total += len(lbl)
        fooled += batch_fool.sum().item()
        
        if batch_fool.sum() > 0:
            diff = (adv - img).reshape(len(img), -1)
            l2 = torch.norm(diff, p=2, dim=1)
            
            examples.append(adv[batch_fool].cpu())
            true_lbls.append(lbl[batch_fool].cpu())
            fooled_lbls.append(pred[batch_fool].cpu())
            dists.append(l2[batch_fool].cpu())

    rob_acc = 100 * (1 - fooled/total)
    avg_dist = torch.cat(dists).mean().item() if dists else 0
    
    torch.save({
        'adv': torch.cat(examples)[:500], 'lbl': torch.cat(true_lbls)[:500],
        'metrics': {'robustness': rob_acc, 'avg_l2': avg_dist}
    }, f"{dirs['std_adv']}/500_adv_{filename}")
    print(f"  Robustness: {rob_acc:.2f}%, Avg L2: {avg_dist:.4f}")

print("\n--- Generating Proxy Attacks (Source = Proxy) ---")
proxies = {
    'lenet': train_proxy_model('lenet'),
    'squeezenet': train_proxy_model('squeezenet')
}

for arch, _ in target_models:
    proxy_model = proxies[arch]
    adversary = PGD(proxy_model, eps=0.3, steps=40, random_start=True)
    
    examples, true_lbls = [], []
    
    for img, lbl in loader:
        if len(examples) >= 500: break
        img, lbl = img.to(device), lbl.to(device)
        adv = adversary(img, lbl)
        
        examples.append(adv.cpu())
        true_lbls.append(lbl.cpu())

    torch.save({
        'adv': torch.cat(examples)[:500], 
        'lbl': torch.cat(true_lbls)[:500]
    }, f"{dirs['proxy_adv']}/proxy_adv_{arch}_for_transfer.pth")
    print(f"  Saved proxy attacks for {arch}")